### Document Edge Detection and Cropping:

In [ ]:
import cv2
import numpy as np

# Load the input image
input_image = cv2.imread('input_document.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Use Canny edge detection to detect edges
edges = cv2.Canny(blurred, 50, 150)

# Find contours in the edges
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the largest contour (assuming it's the document)
largest_contour = max(contours, key=cv2.contourArea)

# Compute the bounding box of the largest contour
x, y, w, h = cv2.boundingRect(largest_contour)

# Crop the document from the input image
cropped_document = input_image[y:y+h, x:x+w]

# Save the cropped document
cv2.imwrite('cropped_document.jpg', cropped_document)


### Document Enhancement:

In [ ]:
def enhance_document(input_image_path, output_image_path):
    # Load the input image
    input_image = cv2.imread(input_image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to enhance text
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Correct skew (deskew)
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    rotated = cv2.warpAffine(input_image, cv2.getRotationMatrix2D((input_image.shape[1] / 2, input_image.shape[0] / 2), angle, 1.0),
                            (input_image.shape[1], input_image.shape[0]), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    # Save the enhanced document
    cv2.imwrite(output_image_path, rotated)

# Usage: enhance_document('cropped_document.jpg', 'enhanced_document.jpg')
